# Airline delays 
## Bureau of Transportation Statistics
https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236   
https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

2015 - 2019

### Additional sources
This might be useful in matching station codes to airports:
1. http://dss.ucar.edu/datasets/ds353.4/inventories/station-list.html
2. https://www.world-airport-codes.com/

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext
from pyspark.sql import types
from pyspark.sql.functions import *
from datetime import datetime, timedelta
from pyspark.sql.window import Window

sqlContext = SQLContext(sc)


In [0]:
username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')
userhome = 'dbfs:/user/' + username
final_project_path = userhome + "/FINAL_PROJECT/" 

In [0]:
FINAL_PROJECT_path_open = '/dbfs' + final_project_path.split(':')[-1] # for use with python open()
dbutils.fs.mkdirs(final_project_path)

Out[37]: True

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/airlines_data/,airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/allstate-claims-severity.zip,allstate-claims-severity.zip,51204863
dbfs:/mnt/mids-w261/datasets_final_project/dac.tar.gz,dac.tar.gz,4576820670
dbfs:/mnt/mids-w261/datasets_final_project/kdd-cup-2014-predicting-excitement-at-donors-choose.zip,kdd-cup-2014-predicting-excitement-at-donors-choose.zip,971133938
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/,parquet_airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_3m/,parquet_airlines_data_3m/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_6m/,parquet_airlines_data_6m/,0
dbfs:/mnt/mids-w261/datasets_final_project/porto-seguro-safe-driver-prediction.zip,porto-seguro-safe-driver-prediction.zip,80247571
dbfs:/mnt/mids-w261/datasets_final_project/walmart-recruiting-trip-type-classification.zip,walmart-recruiting-trip-type-classification.zip,11510035
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/,weather_data/,0


In [0]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/weather_data"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather-miss.parquet/,weather-miss.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2015a.parquet/,weather2015a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2016a.parquet/,weather2016a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2017a.parquet/,weather2017a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2018a.parquet/,weather2018a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2019a.parquet/,weather2019a.parquet/,0


### Pull airline data from parquet on dbfs

In [0]:
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/201*.parquet")
display(airlines.sample(False, 0.00001))

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
2019,2,6,7,5,2019-06-07,AA,19805,AA,N916AN,890,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,13796,1379608,32457,OAK,"Oakland, CA",CA,6,California,91,908,908,0.0,0.0,0.0,0,0900-0959,33.0,941,1051,5.0,1055,1056,1.0,1.0,0.0,0,1000-1059,0.0,null,0.0,227.0,228.0,190.0,1.0,1457.0,6,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,16,7,2019-06-16,DL,19790,DL,N689DL,827,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12,Florida,33,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,630,623,-7.0,0.0,0.0,-1,0600-0659,15.0,638,759,4.0,820,803,-17.0,0.0,0.0,-2,0800-0859,0.0,null,0.0,110.0,100.0,81.0,1.0,581.0,3,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,2,7,2019-06-02,WN,19393,WN,N200WN,3084,13342,1334207,33342,MKE,"Milwaukee, WI",WI,55,Wisconsin,45,15016,1501606,31123,STL,"St. Louis, MO",MO,29,Missouri,64,2000,1953,-7.0,0.0,0.0,-1,2000-2059,9.0,2002,2052,4.0,2115,2056,-19.0,0.0,0.0,-2,2100-2159,0.0,null,0.0,75.0,63.0,50.0,1.0,317.0,2,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,3,1,2019-06-03,WN,19393,WN,N256WN,1300,12892,1289208,32575,LAX,"Los Angeles, CA",CA,6,California,91,12339,1233904,32337,IND,"Indianapolis, IN",IN,18,Indiana,42,1140,1147,7.0,7.0,0.0,0,1100-1159,11.0,1158,1825,5.0,1850,1830,-20.0,0.0,0.0,-2,1800-1859,0.0,null,0.0,250.0,223.0,207.0,1.0,1814.0,8,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,28,5,2019-06-28,WN,19393,WN,N440LV,950,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37,North Carolina,36,10693,1069302,30693,BNA,"Nashville, TN",TN,47,Tennessee

In [0]:
airlines.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- OP_CARRIER_FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- FIRST_DEP_TIME: integer (nullable = true)
-- TOTAL_ADD_GTIME: double (nullable = true)
-- LONGEST_ADD_GTIME: double (nullable = true)
-- DIV_AIRPORT_LANDINGS: integer (nullable = true)
-- DIV_REACHED_DEST: double (nullable = true)
-- DIV_ACTUAL_ELAPSED_TIME: double (nullable = true)
-- DIV_ARR_DELAY: double (nullable = true)
-- DIV_DISTANCE: double (nullable = true)
-- DIV1_AIRPORT: string (nullable = true)
-- DIV1_AIRPORT_ID: integer (nullable = true)
-- DIV1_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV1_WHEELS_ON: integer (nullable = true)
-- DIV1_TOTAL_GTIME: double (nullable = true)
-- DIV1_LONGEST_GTIME: double (nullable = true)
-- DIV1_WHEELS_OFF: integer (nullable = true)
-- DIV1_TAIL_NUM: string (nullable = true)
-- DIV2_AIRPORT: string (nullable = true)
-- DIV2_AIRPORT_ID: integer (nullable = true)
-- DIV2_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV2_WHEELS_ON: integer (nullable = true)
-- DIV2_TOTAL_GTIME: double (nullable = true)
-- DIV2_LONGEST_GTIME: double (nullable = true)
-- DIV2_WHEELS_OFF: integer (nullable = true)
-- DIV2_TAIL_NUM: string (nullable = true)
-- DIV3_AIRPORT: string (nullable = true)
-- DIV3_AIRPORT_ID: integer (nullable = true)
-- DIV3_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV3_WHEELS_ON: integer (nullable = true)
-- DIV3_TOTAL_GTIME: double (nullable = true)
-- DIV3_LONGEST_GTIME: double (nullable = true)
-- DIV3_WHEELS_

In [0]:
f'{airlines.count():,}'

Out[11]: '31,746,841'

### Pull timezone reference chart from csv on dbfs

In [0]:
city_timezone = spark.read.option("header", "false").csv("dbfs:/FileStore/tables/cities1000-4.csv")

# Weather and Join Work
https://data.nodc.noaa.gov/cgi-bin/iso?id=gov.noaa.ncdc:C00532

#### Pulling and filtering data

In [0]:
#All Weather Files
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/weather_data"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather-miss.parquet/,weather-miss.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2015a.parquet/,weather2015a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2016a.parquet/,weather2016a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2017a.parquet/,weather2017a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2018a.parquet/,weather2018a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2019a.parquet/,weather2019a.parquet/,0


In [0]:
#All Weather Files
weather = spark.read.option("header", "true")\
                    .parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/weather_data/*.parquet")

f'{weather.count():,}'

Out[14]: '630,904,436'

In [0]:
# WEATHER ONLY 2015
weather2015 = spark.read.option("header", "true")\
                    .parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2015a.parquet")

f'{weather2015.count():,}'

Out[15]: '116,341,048'

In [0]:
def create_airport_code_stations(call_code):
    try:
      if call_code[0] == 'K':
        airport_code = call_code[1:4]
      else:
        airport_code = ''
    except:
      airport_code = ''
    return airport_code

create_airport_code_stations_udf = f.udf(create_airport_code_stations, types.StringType())
weather_q12015_coded = weather_q12015.withColumn("airport_code", create_airport_code_stations_udf('CALL_SIGN')) 

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-945005255680309> in <module> 
 10 
 11 create_airport_code_stations_udf = f . udf ( create_airport_code_stations , types . StringType ( ) ) 
 ---> 12 weather_q12015_coded = weather_q12015 . withColumn ( "airport_code" , create_airport_code_stations_udf ( 'CALL_SIGN' ) ) 

 NameError : name 'weather_q12015' is not defined

#### FILTER WEATHER TO Q1 2015. SAVE FILE TO DBFS

In [0]:
#FILTER on Q1 and Airport stations
weather_atl_ord_q12015 = weather2015.filter((weather2015_atl_ord.DATE <= "2015-03-31") & 
                            ((weather2015.NAME == "CHICAGO OHARE INTERNATIONAL AIRPORT, IL US") | \
                            (weather2015.NAME == "ATLANTA HARTSFIELD INTERNATIONAL AIRPORT, GA US")))
#WILL WANT TO TAKE OUT THE AIRPORT FILTER LATER SINCE WE NEED TO BRING IN WEATHER AT DESTINATION STATIONS AS WELL

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-945005255680311> in <module> 
 1 #FILTER on Q1 and Airport stations 
 ----> 2 weather_atl_ord_q12015 = weather2015.filter((weather2015_atl_ord.DATE <= "2015-03-31") & 
 3 ((weather2015.NAME == "CHICAGO OHARE INTERNATIONAL AIRPORT, IL US") | \
 4 (weather2015.NAME == "ATLANTA HARTSFIELD INTERNATIONAL AIRPORT, GA US")))
 5 #WILL WANT TO TAKE OUT THE AIRPORT FILTER LATER SINCE WE NEED TO BRING IN WEATHER AT DESTINATION STATIONS AS WELL 

 NameError : name 'weather2015_atl_ord' is not defined

In [0]:
#SAVE FILE TO DBFS
weather_atl_ord_q12015.write.format("parquet").save(final_project_path + "weather_atl_ord_q1_2015.parquet")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-945005255680312> in <module> 
 1 #SAVE FILE TO DBFS 
 ----> 2 weather_atl_ord_q12015 . write . format ( "parquet" ) . save ( final_project_path + "weather_atl_ord_q1_2015.parquet" ) 

 NameError : name 'weather_atl_ord_q12015' is not defined

In [0]:
#FILTER on Q1
weather_q12015 = weather2015.filter(weather2015.DATE <= "2015-03-31")

##### Add Airport code to weather data

In [0]:
def create_airport_code_stations(call_code):
    try:
      if call_code[0] == 'K':
        airport_code = call_code[1:4]
      else:
        airport_code = ''
    except:
      airport_code = ''
    return airport_code

create_airport_code_stations_udf = f.udf(create_airport_code_stations, types.StringType())
weather_q12015_coded = weather_q12015.withColumn("airport_code", create_airport_code_stations_udf('CALL_SIGN')) 

In [0]:
#SAVE FILE TO DBFS
weather_q12015_coded.write.format("parquet").save(final_project_path + "weather_q12015_coded.parquet")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-945005255680316> in <module> 
 1 #SAVE FILE TO DBFS 
 ----> 2 weather_q12015_coded . write . format ( "parquet" ) . save ( final_project_path + "weather_q12015_coded.parquet" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 828 self . _jwrite . save ( ) 
 829 else : 
 --> 830 self . _jwrite . save ( path ) 
 831 
 832 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : path dbfs:/user/robin.lashof@ischool.berkeley.edu/FINAL_PROJECT/weather_q12015_coded.parquet already exists.;

#### FILTER FLIGHTS TO Q1 2015, DEPARTING ORD AND ATL. SAVE FILE TO DBFS

In [0]:
#FILTER on Q1 and Airport stations
airlines_ATL_ORD_Q12015 = airlines.filter( (airlines.QUARTER  == 1) & (airlines.YEAR  == 2015) & \
               ((airlines.ORIGIN  == 'ORD') | (airlines.ORIGIN  == 'ATL')))

In [0]:
def split_city_name(city_state):
  '''UDF to deal with cases where dual cities are labeled
  with a "/". Returns only first city '''

  city = city_state.split(',')[0]
  state = city_state.split(',')[1]
  shortened_city = city.split('/')[0]
  
  return shortened_city + ',' + state

split_city_name_udf = f.udf(split_city_name, types.StringType())

airlines_ATL_ORD_Q12015_short_city = airlines_ATL_ORD_Q12015.withColumn("SHORT_DEST_CITY_NAME", split_city_name_udf('DEST_CITY_NAME'))

In [0]:
#SAVE FILE TO DBFS
airlines_ATL_ORD_Q12015_short_city.write.format("parquet").save(final_project_path + "airlines_ATL_ORD_Q12015_mod.parquet")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-945005255680320> in <module> 
 1 #SAVE FILE TO DBFS 
 ----> 2 airlines_ATL_ORD_Q12015_short_city . write . format ( "parquet" ) . save ( final_project_path + "airlines_ATL_ORD_Q12015_mod.parquet" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 828 self . _jwrite . save ( ) 
 829 else : 
 --> 830 self . _jwrite . save ( path ) 
 831 
 832 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : path dbfs:/user/robin.lashof@ischool.berkeley.edu/FINAL_PROJECT/airlines_ATL_ORD_Q12015_mod.parquet already exists.;

#### READ FROM SAVED FILES ---START FROM HERE---
note: still need to run first few cells to connect to dbfs and also the cell that reads in city_timezone

In [0]:
weather_q12015_coded = spark.read.parquet(final_project_path+"weather_q12015_coded.parquet")

In [0]:
airlines_ATL_ORD_Q12015_saved = spark.read.parquet(final_project_path+"airlines_ATL_ORD_Q12015_mod.parquet")

#### Round Airline Scheduled time and Weather data to nearest hour

In [0]:
city_timezone.createOrReplaceTempView("city_timezone")
sqlContext.sql("""
CREATE TEMPORARY VIEW city_state_timezone
AS
SELECT 
  _c0 AS city_id,
  _c1 AS city,
  _c2 AS country,
  _c3 AS state,
  _c4 AS timezone,
  CONCAT(_c1, ', ', _c3) AS city_state
FROM city_timezone
""")

Out[8]: DataFrame[]

In [0]:
airlines_ATL_ORD_Q12015_saved.createOrReplaceTempView("atl_ord_q12015")
sqlContext.sql("""
DROP VIEW IF EXISTS atl_ord_q12015_with_timezones
""")
sqlContext.sql("""
CREATE TEMPORARY VIEW atl_ord_q12015_with_timezones
AS
SELECT
  year,
  quarter,
  month,
  day_of_month,
  day_of_week,
  fl_date,
  op_unique_carrier,
  op_carrier_airline_id,
  op_carrier,
  tail_num,
  op_carrier_fl_num,
  origin_airport_id,
  origin_airport_seq_id,
  origin_city_market_id,
  origin,
  origin_city_name,
  origin_state_abr,
  origin_state_fips,
  origin_state_nm,
  origin_wac,
  dest_airport_id,
  dest_airport_seq_id,
  dest_city_market_id,
  dest,
  dest_city_name,
  dest_state_abr,
  dest_state_fips,
  dest_state_nm,
  dest_wac,
  crs_dep_time,
  dep_time,
  dep_delay,
  dep_delay_new,
  dep_del15,
  dep_delay_group,
  dep_time_blk,
  taxi_out,
  wheels_off,
  wheels_on,
  taxi_in,
  crs_arr_time,
  arr_time,
  arr_delay,
  arr_delay_new,
  arr_del15,
  arr_delay_group,
  arr_time_blk,
  cancelled,
  cancellation_code,
  diverted,
  crs_elapsed_time,
  actual_elapsed_time,
  air_time,
  flights,
  distance,
  distance_group,
  carrier_delay,
  weather_delay,
  nas_delay,
  security_delay,
  late_aircraft_delay,
  short_dest_city_name,
  td.timezone AS dest_timezone,
  to.timezone AS origin_timezone,
  TO_UTC_TIMESTAMP(DATE_TRUNC('hour', TO_TIMESTAMP(CONCAT(fl_date, ' ', crs_dep_time), 'yyyy-MM-dd Hmm')), to.timezone) AS truncated_crs_dep_time_utc,
  TO_UTC_TIMESTAMP(DATE_TRUNC('hour', TO_TIMESTAMP(CONCAT(fl_date, ' ', crs_dep_time), 'yyyy-MM-dd Hmm')), to.timezone) - INTERVAL 3 HOURS AS truncated_crs_dep_minus_three_utc
FROM atl_ord_q12015 AS f
LEFT JOIN city_state_timezone AS td ON
  f.short_dest_city_name = td.city_state
LEFT JOIN city_state_timezone AS to ON
  f.origin_city_name = to.city_state
""")
# NOTES:
# Do we want to filter out DEP_TIME=null?
# Definitely don't need all these columns.
# Should joins be inner? and maybe we need to check the counts to make sure it's joining up properly?


Out[32]: DataFrame[]

In [0]:
display(sqlContext.sql("DESC FORMATTED atl_ord_q12015_with_timezones"))

col_name,data_type,comment
year,int,null
quarter,int,null
month,int,null
day_of_month,int,null
day_of_week,int,null
fl_date,string,null
op_unique_carrier,string,null
op_carrier_airline_id,int,null
op_carrier,string,null
tail_num,string,null


In [0]:
display(sqlContext.sql("SELECT * FROM atl_ord_q12015_with_timezones LIMIT 10"))

year,quarter,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_airline_id,op_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin_airport_seq_id,origin_city_market_id,origin,origin_city_name,origin_state_abr,origin_state_fips,origin_state_nm,origin_wac,dest_airport_id,dest_airport_seq_id,dest_city_market_id,dest,dest_city_name,dest_state_abr,dest_state_fips,dest_state_nm,dest_wac,crs_dep_time,dep_time,dep_delay,dep_delay_new,dep_del15,dep_delay_group,dep_time_blk,dep_time_blk,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,arr_delay_new,arr_del15,arr_delay_group,arr_time_blk,cancelled,cancellation_code,diverted,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,short_dest_city_name,dest_timezone,origin_timezone,truncated_crs_dep_time_utc,truncated_crs_dep_minus_three_utc
2015,1,2,1,7,2015-02-01,AA,19805,AA,N3MEAA,392,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,730,749,19.0,19.0,1.0,1,0700-0759,0700-0759,34.0,823,1056,4.0,1030,1100,30.0,30.0,1.0,2,1000-1059,0.0,null,0.0,120.0,131.0,93.0,1.0,733.0,3,0.0,19.0,11.0,0.0,0.0,"New York, NY",America/New_York,America/Chicago,2015-02-01T13:00:00.000+0000,2015-02-01T10:00:00.000+0000
2015,1,2,2,1,2015-02-02,AA,19805,AA,N3LPAA,392,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,730,null,null,null,null,null,0700-0759,0700-0759,null,null,null,null,1030,null,null,null,null,null,1000-1059,1.0,B,0.0,120.0,null,null,1.0,733.0,3,null,null,null,null,null,"New York, NY",America/New_York,America/Chicago,2015-02-02T13:00:00.000+0000,2015-02-02T10:00:00.000+0000
2015,1,2,3,2,2015-02-03,AA,19805,AA,N3LXAA,392,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,730,735,5.0,5.0,0.0,0,0700-0759,0700-0759,19.0,754,1027,9.0,1030,1036,6.0,6.0,0.0,0,1000-1059,0.0,null,0.0,120.0,121.0,93.0,1.0,733.0,3,null,null,null,null,null,"New York, NY",America/New_York,America/Chicago,2015-02-03T13:00:00.000+0000,2015-02-03T10:00:00.000+0000
2015,1,2,4,3,2015-02-04,AA,19805,AA,N3LHAA,392,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,730,727,-3.0,0.0,0.0,-1,0700-0759,0700-0759,11.0,738,1008,5.0,1030,1013,-17.0,0.0,0.0,-2,1000-1059,0.0,null,0.0,120.0,106.0,90.0,1.0,733.0,3,null,null,null,null,null,"New York, NY",America/New_York,America/Chicago,2015-02-04T13:00:00.000+0000,2015-02-04T10:00:00.000+0000
2015,1,2,5,4,2015-02-05,AA,19805,AA,N3LYAA,392,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,730,757,27.0,27.0,1.0,1,0700-0759,0700-0759,23.0,820,1058,3.0,1030,1101,31.0,31.0,1.0,2,1000-1059,0.0,null,0.0,120.0,124.0,98.0,1.0,733.0,3,0.0,0.0,31.0,0.0,0.0,"New York, NY",America/New_York,America/Chicago,2015-02-05T13:00:00.000+0000,2015-02-05T10:00:00.000+0000
2015,1,2,6,5,2015-02-06,AA,19805,AA,N3LVAA,392,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,730,723,-7.0,0.0,0.0,-1,0700-0759,0700-0759,10.0,733,1007,4.0,1030,1011,-19.0,0.0,0.0,-2,1000-1059,0.0,null,0.0,120.0,108.0,94.0,1.0,733.0,3,null,null,null,null,null,"New York, NY",America/New_York,America/Chicago,2015-02-06T13:00:00.000+0000,2015-02-06T10:00:00.000+0000
2015,1,2,8,7,2015-02-08,AA,19805,AA,N3MAAA,392,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,12953,1295302,31703,LGA,"New York, NY",NY,36,New York,22,730,728,-2.0,0.0,0.0,-1,0700-0759,0700-0759,10.0,738,1012,6.0,1030,1018,-12.0,0.0,0.0,-1,1000-1059,0.0,null,0.0,120.0,110.0,94.0,1.0,733.0,3,null,null,null,null,null,"New York, NY",America/New_York,America/Chicago,2015-02-08T13:00:00.000+0000,2015-02-08T10:00:00.000+0000
2015,1,2,9,1,2015-02-09,AA,19805,AA,N3MGAA,392,13930,13

In [0]:
display(weather_q12015_coded.limit(10))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3,airport_code
47739099999,2015-01-01T00:00:00.000+0000,4,34.8,138.1833333,135.0,"SHIZUOKA AIRPORT, JA",FM-15,99999,V020,"260,1,N,0098,1","99999,9,9,N","009999,1,9,9","+0040,1","-0030,1","99999,9",,"02,1,+00610,1,99,9",,,,"9,AGL ,+99999,+99999","99,99,9,02,1,99,9,00610,1,99,9,99,9",,,"10040,1,99999,9",,,,,,,MET069METAR RJNS 010000Z 26019KT 9999 FEW020 04/M03 Q1004 RMK 1CU020 A2967=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
47739099999,2015-01-01T00:03:00.000+0000,4,34.8,138.1833333,135.0,"SHIZUOKA AIRPORT, JA",FM-16,99999,V020,"260,1,N,0093,1","99999,9,9,N","009999,1,9,9","+0040,1","-0030,1","99999,9",,"02,1,+00610,1,99,9",,,,"9,AGL ,+99999,+99999","99,99,9,02,1,99,9,00610,1,99,9,99,9",,,"10040,1,99999,9",,,,"0144,1",,,MET072SPECI RJNS 010003Z 26018G28KT 9999 FEW020 04/M03 Q1004 RMK 1CU020 A2967=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
47739099999,2015-01-01T01:00:00.000+0000,4,34.8,138.1833333,135.0,"SHIZUOKA AIRPORT, JA",FM-15,99999,V020,"260,1,N,0108,1","99999,9,9,N","009999,1,9,9","+0040,1","-0030,1","99999,9",,"02,1,+00610,1,99,9",,,,"9,AGL ,+99999,+99999","99,99,9,02,1,99,9,00610,1,99,9,99,9",,,"10040,1,99999,9",,,,"0165,1",,,MET072METAR RJNS 010100Z 26021G32KT 9999 FEW020 04/M03 Q1004 RMK 1CU020 A2967=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
47739099999,2015-01-01T02:00:00.000+0000,4,34.8,138.1833333,135.0,"SHIZUOKA AIRPORT, JA",FM-15,99999,V020,"260,1,N,0118,1","99999,9,9,N","009999,1,9,9","+0050,1","-0060,1","99999,9",,"02,1,+00610,1,99,9",,,,"9,AGL ,+99999,+99999","99,99,9,02,1,99,9,00610,1,99,9,99,9",,,"10030,1,99999,9",,,,,,,MET069METAR RJNS 010200Z 26023KT 9999 FEW020 05/M06 Q1003 RMK 2CU020 A2964=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
47739099999,2015-01-01T02:02:00.000+0000,4,34.8,138.1833333,135.0,"SHIZUOKA AIRPORT, JA",FM-16,99999,V020,"260,1,N,0118,1","99999,9,9,N","009999,1,9,9","+0050,1","-0050,1","99999,9",,"02,1,+00610,1,99,9",,,,"9,AGL ,+99999,+99999","99,99,9,02,1,99,9,00610,1,99,9,99,9",,,"10030,1,99999,9",,,,"0170,1",,,MET072SPECI RJNS 010202Z 26023G33KT 9999 FEW020 05/M05 Q1003 RMK 2CU020 A2964=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
47739099999,2015-01-01T03:00:00.000+0000,4,34.8,138.1833333,135.0,"SHIZUOKA AIRPORT, JA",FM-15,99999,V020,"270,1,N,0108,1","01372,1,C,N","009999,1,9,9","+0050,1","-0100,1","99999,9",,"02,1,+00610,1,99,9","04,1,+01372,1,99,9",,,"9,AGL ,+99999,+99999","99,99,9,02,1,99,9,00610,1,99,9,99,9",,,"10020,1,99999,9",,,,,,,MET093METAR RJNS 010300Z 27021KT 9999 FEW020 SCT045 05/M10 Q1002 RMK 1CU020 3CU045 A2959 P/FR=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
47739099999,2015-01-01T03:04:00.000+0000,4,34.8,138.1833333,135.0

In [0]:
#ADD TRUNCATED UTC TIME TO WEATHER DATA DOWN TO HOUR
weather_q12015_coded.createOrReplaceTempView("weather_q12015_coded")
sqlContext.sql("""
DROP VIEW IF EXISTS weather_q12015
""")
sqlContext.sql("""
CREATE TEMPORARY VIEW weather_q12015
AS
WITH weather_temp
AS
(
  SELECT
    wc.*,
    DATE_TRUNC('hour', TO_TIMESTAMP(wc.date, "yyyy-MM-ddTHH:mm:ss 'UTC'")) AS hour
  FROM weather_q12015_coded AS wc
),
weather_ranked
AS
(
  SELECT
    wt.*,
    ROW_NUMBER() OVER(PARTITION BY wt.hour, wt.airport_code ORDER BY wt.date) as rank
  FROM weather_temp AS wt
)
SELECT
  wr.station,
  wr.date,
  wr.source,
  wr.latitude,
  wr.longitude,
  wr.elevation,
  wr.name,
  wr.report_type,
  wr.quality_control,
  wr.wnd,
  wr.cig,
  wr.vis,
  wr.tmp,
  wr.dew,
  wr.slp,
  wr.ga1,
  wr.ge1,
  wr.gf1,
  wr.ma1,
  wr.rem,
  wr.gd1,
  wr.aa1,
  wr.airport_code,
  wr.hour
FROM weather_ranked AS wr
WHERE
  wr.rank = 1
""")


Out[26]: DataFrame[]

In [0]:
display(sqlContext.sql("SELECT * FROM weather_q12015 LIMIT 10"))

station,date,source,latitude,longitude,elevation,name,report_type,quality_control,wnd,cig,vis,tmp,dew,slp,ga1,ge1,gf1,ma1,rem,gd1,aa1,airport_code,hour
72203812897,2015-01-01T00:51:00.000+0000,7,26.155,-81.7752,2.7,"NAPLES MUNICIPAL AIRPORT, FL US",FM-16,V030,"020,5,N,0031,5","00549,5,M,N","016093,5,N,5","+0220,5","+0190,5","99999,9","07,5,+00549,5,99,9","9,AGL ,+99999,+99999","99,99,9,99,9,99,9,00549,1,99,9,99,9","10227,5,10218,5",MET08112/31/14 19:51:03 SPECI KAPF 010051Z 02006KT 10SM BKN018 22/19 A3020 RMK AO2 (DR),"3,99,1,+00549,5,9",,APF,2015-01-01T00:00:00.000+0000
74465504808,2015-01-01T00:52:00.000+0000,7,41.77,-88.48139,216.4,"CHICAGO AURORA MUNICIPAL AIRPORT, IL US",FM-15,V020,"230,5,N,0067,5","22000,5,9,N","016093,5,N,5","-0083,5","-0167,5","10249,5","00,5,+99999,9,99,9",,"00,99,1,99,9,99,9,99999,9,99,9,99,9","10234,5,09972,5",MET09912/31/14 18:52:03 METAR KARR 010052Z 23013KT 10SM CLR M08/M17 A3022 RMK AO2 SLP249 T10831167 $ (EM),"0,99,1,+99999,9,9","01,0000,9,5",ARR,2015-01-01T00:00:00.000+0000
72392623136,2015-01-01T00:55:00.000+0000,7,34.21667,-119.08333,23.5,"CAMARILLO AIRPORT, CA US",FM-15,V020,"290,5,N,0026,5","22000,5,9,N","016093,5,N,5","+0089,5","-0011,5","10181,5","00,5,+99999,9,99,9",,"00,99,1,99,9,99,9,99999,9,99,9,99,9","10173,5,10149,5",MET09612/31/14 16:55:03 METAR KCMA 010055Z 29005KT 10SM CLR 09/M01 A3004 RMK AO2 SLP181 T00891011 (RC),"0,99,1,+99999,9,9","01,0000,9,5",CMA,2015-01-01T00:00:00.000+0000
72325003847,2015-01-01T00:53:00.000+0000,7,35.9509,-85.0813,569.1,"CROSSVILLE MEMORIAL AIRPORT, TN US",FM-15,V030,"999,9,C,0000,5","22000,5,9,N","016093,5,N,5","-0017,5","-0100,5","10306,5","00,5,+99999,9,99,9",,"00,99,1,99,9,99,9,99999,9,99,9,99,9","10288,5,09613,5",MET09412/31/14 18:53:03 METAR KCSV 010053Z 00000KT 10SM CLR M02/M10 A3038 RMK AO2 SLP306 T10171100 $,"0,99,1,+99999,9,9","01,0000,9,5",CSV,2015-01-01T00:00:00.000+0000
72438453842,2015-01-01T00:53:00.000+0000,7,39.825,-86.29583,250.9,"INDIANAPOLIS EAGLE CREEK AIRPORT, IN US",FM-15,V020,"220,5,N,0031,5","22000,5,9,N","016093,5,N,5","-0067,5","-0144,5","10285,5","00,5,+99999,9,99,9",,"00,99,1,99,9,99,9,99999,9,99,9,99,9","10271,5,09970,5",MET09212/31/14 19:53:03 METAR KEYE 010053Z 22006KT 10SM CLR M07/M14 A3033 RMK AO2 SLP285 T10671144,"0,99,1,+99999,9,9","01,0000,9,5",EYE,2015-01-01T00:00:00.000+0000
72317013723,2015-01-01T00:54:00.000+0000,7,36.0969,-79.9432,271.3,"GREENSBORO AIRPORT, NC US",FM-15,V030,"230,5,N,0021,5","22000,5,9,N","016093,5,N,5","+0000,5","-0061,5","10281,5","02,5,+07620,5,99,9","9,AGL ,+99999,+99999","02,99,5,99,9,99,9,07620,1,99,9,99,9","10274,5,09950,5",MET10012/31/14 19:54:03 METAR KGSO 010054Z 23004KT 10SM FEW250 00/M06 A3034 RMK AO2 SLP281 T00001061 (JTS),"1,99,1,+07620,5,9","01,0000,9,5",GSO,2015-01-01T00:00:00.000+0000
72242712975,2015-01-01T00:53:00.000+0000,7,29.51889,-95.24167,13.4,"HOUSTON CLOVER FIELD, TX US",FM-15,V020,"030,5,N,0057,5","02743,5,M,N","016093,5,N,5","+0083,5","+0017,5","10281,5","08,5,+02743,5,99,9","9,AGL ,+99999,+99999","99,99,9,99,9,99,9,02743,1,99,9,99,9","10284,5,10269,5",MET09612/31/14 18:53:03 METAR KLVJ 010053Z 03011G19KT 10SM OVC090 08/02 A3037 RMK AO2 SLP281 T00830017,"4,99,1,+02743,5,9","01,0000,9,5",LVJ,2015-01-01T00:00:00.000+0000
72681724154,2015-01-01T00:53:00.000+0000,7,47.45694,-115.645,1837.3,"MULLAN PASS VOR DME, ID US",FM-15,V020,"240,5,N,0015,5","22000,5,9,N","016093,5,N,5","-0144,5","-0172,5","10340,5","00,5,+99999,9,99,9",,"00,99,1,99,9,99,9,99999,9,99,9,99,9","10227,5,08175,5",MET09212/31/14 16:53:03 METAR KMLP 010053Z 24003KT 10SM CLR M14/M17 A3020 RMK AO2 SLP340 T11441172,"0,99,1,+99999,9,9","01,0000,9,5",MLP,2015-01-01T00:00:00.000+0000
72650014972,2015-01-01T00:53:00.000+0000,7,43.16444,-95.20167,407.8,"SPENCER MUNICIPAL AIRPORT, IA US",FM-15,V020,"260,5,N,0062,5","22000,5,9,N","016093,5,N,5","-0111,5","-0167,5","10242,5","00,5,+99999,9,99,9",,"00,99,1,99,9,99,9,99999,9,99,9,99,9","10210,5,09726,5",MET09712/31/14 18:53:03 METAR KSPW 010053Z 26012KT 10SM CLR M11/M17

#### JOIN FLIGHT AND WEATHER DATA

In [0]:
#JOIN WEATHER AND AIRLINE DATA FOR ORIGIN BETWEEN 2-3 hours DEFORE DEPARTURE
weather_airline_joined_full = sqlContext.sql("""
SELECT
  f.*,
  wo.station AS origin_weather_station,
  wo.date AS origin_weather_date,
  wo.source AS origin_weather_source,
  wo.latitude AS origin_weather_latitude,
  wo.longitude AS origin_weather_longitude,
  wo.elevation AS origin_weather_elevation,
  wo.name AS origin_weather_name,
  wo.report_type AS origin_weather_report_type,
  wo.quality_control AS origin_weather_quality_control,
  wo.wnd AS origin_weather_wnd,
  wo.cig AS origin_weather_cig,
  wo.vis AS origin_weather_vis,
  wo.tmp AS origin_weather_tmp,
  wo.dew AS origin_weather_dew,
  wo.slp AS origin_weather_slp,
  wo.ga1 AS origin_weather_ga1,
  wo.ge1 AS origin_weather_ge1,
  wo.gf1 AS origin_weather_gf1,
  wo.ma1 AS origin_weather_ma1,
  wo.rem AS origin_weather_rem,
  wo.gd1 AS origin_weather_gd1,
  wo.aa1 AS origin_weather_aa1,
  wo.airport_code AS origin_weather_airport_code,
  wo.hour AS origin_weather_hour,
  wd.station AS dest_weather_station,
  wd.date AS dest_weather_date,
  wd.source AS dest_weather_source,
  wd.latitude AS dest_weather_latitude,
  wd.longitude AS dest_weather_longitude,
  wd.elevation AS dest_weather_elevation,
  wd.name AS dest_weather_name,
  wd.report_type AS dest_weather_report_type,
  wd.quality_control AS dest_weather_quality_control,
  wd.wnd AS dest_weather_wnd,
  wd.cig AS dest_weather_cig,
  wd.vis AS dest_weather_vis,
  wd.tmp AS dest_weather_tmp,
  wd.dew AS dest_weather_dew,
  wd.slp AS dest_weather_slp,
  wd.ga1 AS dest_weather_ga1,
  wd.ge1 AS dest_weather_ge1,
  wd.gf1 AS dest_weather_gf1,
  wd.ma1 AS dest_weather_ma1,
  wd.rem AS dest_weather_rem,
  wd.gd1 AS dest_weather_gd1,
  wd.aa1 AS dest_weather_aa1,
  wd.airport_code AS dest_weather_airport_code,
  wd.hour AS dest_weather_hour
FROM atl_ord_q12015_with_timezones AS f
LEFT JOIN weather_q12015 AS wo ON
  f.origin = wo.airport_code
  AND f.truncated_crs_dep_minus_three_utc = wo.hour
LEFT JOIN weather_q12015 AS wd ON
  f.dest = wd.airport_code
  AND f.truncated_crs_dep_minus_three_utc = wd.hour
""")

In [0]:
display(weather_airline_joined_full.limit(10))

year,quarter,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_airline_id,op_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin_airport_seq_id,origin_city_market_id,origin,origin_city_name,origin_state_abr,origin_state_fips,origin_state_nm,origin_wac,dest_airport_id,dest_airport_seq_id,dest_city_market_id,dest,dest_city_name,dest_state_abr,dest_state_fips,dest_state_nm,dest_wac,crs_dep_time,dep_time,dep_delay,dep_delay_new,dep_del15,dep_delay_group,dep_time_blk,dep_time_blk,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,arr_delay_new,arr_del15,arr_delay_group,arr_time_blk,cancelled,cancellation_code,diverted,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,short_dest_city_name,dest_timezone,origin_timezone,truncated_crs_dep_time_utc,truncated_crs_dep_minus_three_utc,origin_weather_station,origin_weather_date,origin_weather_source,origin_weather_latitude,origin_weather_longitude,origin_weather_elevation,origin_weather_name,origin_weather_report_type,origin_weather_quality_control,origin_weather_wnd,origin_weather_cig,origin_weather_vis,origin_weather_tmp,origin_weather_dew,origin_weather_slp,origin_weather_ga1,origin_weather_ge1,origin_weather_gf1,origin_weather_ma1,origin_weather_rem,origin_weather_gd1,origin_weather_aa1,origin_weather_airport_code,origin_weather_hour,dest_weather_station,dest_weather_date,dest_weather_source,dest_weather_latitude,dest_weather_longitude,dest_weather_elevation,dest_weather_name,dest_weather_report_type,dest_weather_quality_control,dest_weather_wnd,dest_weather_cig,dest_weather_vis,dest_weather_tmp,dest_weather_dew,dest_weather_slp,dest_weather_ga1,dest_weather_ge1,dest_weather_gf1,dest_weather_ma1,dest_weather_rem,dest_weather_gd1,dest_weather_aa1,dest_weather_airport_code,dest_weather_hour
2015,1,2,22,7,2015-02-22,DL,19790,DL,N995AT,2582,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,10135,1013503,30135,ABE,"Allentown/Bethlehem/Easton, PA",PA,42,Pennsylvania,23,2038,2137,59.0,59.0,1.0,3,2000-2059,2000-2059,20.0,2157,2326,4.0,2230,2330,60.0,60.0,1.0,4,2200-2259,0.0,null,0.0,112.0,113.0,89.0,1.0,692.0,3,59.0,0.0,1.0,0.0,0.0,"Allentown, PA",America/New_York,America/New_York,2015-02-23T01:00:00.000+0000,2015-02-22T22:00:00.000+0000,72219013874,2015-02-22T22:03:00.000+0000,7,33.6301,-84.4418,307.8,"ATLANTA HARTSFIELD INTERNATIONAL AIRPORT, GA US",FM-16,V030,"340,5,N,0031,5","00122,5,M,N","000805,A,N,A","+0130,5","+0110,5","99999,9","08,5,+00122,5,99,9","9,AGL ,+99999,+99999","99,99,9,99,9,99,9,00122,1,99,9,99,9","10207,5,09834,5",MET13402/22/15 17:03:02 SPECI KATL 222203Z 34006KT 1/2SM R09R/6000VP6000FT -DZ BR OVC004 13/11 A3014 RMK AO2 SFC VIS 1 1/2 DZB03 P0000 (KAF),"4,99,1,+00122,5,9","01,0000,2,1",ATL,2015-02-22T22:00:00.000+0000,72517014737,2015-02-22T22:51:00.000+0000,7,40.64985,-75.44771,118.9,"ALLENTOWN LEHIGH VALLEY INTERNATIONAL AIRPORT, PA US",FM-15,V030,"220,5,N,0031,5","22000,5,9,N","009656,5,N,5","+0028,5","-0022,5","10217,5","00,5,+99999,9,99,9",,"00,99,1,99,9,99,9,99999,9,99,9,99,9","10213,5,10073,5",MET09902/22/15 17:51:02 METAR KABE 222251Z 22006KT 6SM HZ CLR 03/M02 A3016 RMK AO2 SLP217 T00281022 (ABE),"0,99,1,+99999,9,9","01,0000,9,5",ABE,2015-02-22T22:00:00.000+0000
2015,1,3,15,7,2015-03-15,MQ,20398,MQ,N522MQ,3157,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,10140,1014003,30140,ABQ,"Albuquerque, NM",NM,35,New Mexico,86,1025,1039,14.0,14.0,0.0,0,1000-1059,1000-1059,20.0,1059,1215,7.0,1233,1222,-11.0,0.0,0.0,-1,1200-1259,0.0,null,0.0,188.0,163.0,136.0,1.0,1118.0,5,null,null,null,null,null,"Albuquerque, NM",America/Denver,America/Chicago,2015-03-15T15:00:00.000+0000,2015-03-15T12:00:00.000+0000,72530094846,2015-03-15T12:51:00.000+0000,7,41.995,-87.9336,201.8,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",FM-15,V030,"220,5,N,0026,5","22000,5,9,N","016093,5,N,5","+0011,5","-0011,5","10248,5","02,5,+07620,5,99,9","9,AG

In [0]:
#SAVE JOINED FILE
weather_airline_joined_full.write.format("parquet").save(final_project_path + "weather_airline_joined.parquet")

(flights_q12015_train, flights_q12015_validation, flights_q12015_test) = weather_airline_joined_full.randomSplit([0.7, 0.15, 0.15], seed=42)

# Save train, test, and eval files
flights_q12015_train.write.format("parquet").save(final_project_path + "flights_q12015_train.parquet")
flights_q12015_validation.write.format("parquet").save(final_project_path + "flights_q12015_validation.parquet")
flights_q12015_test.write.format("parquet").save(final_project_path + "flights_q12015_test.parquet")

In [0]:
#READ BACK TRAIN FILE FOR EDA
weather_airline_joined = spark.read.parquet(final_project_path+"flights_q12015_train.parquet")

In [0]:
display(weather_airline_joined.limit(10))

year,quarter,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_airline_id,op_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin_airport_seq_id,origin_city_market_id,origin,origin_city_name,origin_state_abr,origin_state_fips,origin_state_nm,origin_wac,dest_airport_id,dest_airport_seq_id,dest_city_market_id,dest,dest_city_name,dest_state_abr,dest_state_fips,dest_state_nm,dest_wac,crs_dep_time,dep_time,dep_delay,dep_delay_new,dep_del15,dep_delay_group,dep_time_blk,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,arr_delay_new,arr_del15,arr_delay_group,arr_time_blk,cancelled,cancellation_code,diverted,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,short_dest_city_name,dest_timezone,origin_timezone,truncated_crs_dep_time_utc,truncated_crs_dep_minus_three_utc,origin_weather_station,origin_weather_date,origin_weather_source,origin_weather_latitude,origin_weather_longitude,origin_weather_elevation,origin_weather_name,origin_weather_report_type,origin_weather_quality_control,origin_weather_wnd,origin_weather_cig,origin_weather_vis,origin_weather_tmp,origin_weather_dew,origin_weather_slp,origin_weather_ga1,origin_weather_ge1,origin_weather_gf1,origin_weather_ma1,origin_weather_rem,origin_weather_gd1,origin_weather_aa1,origin_weather_airport_code,origin_weather_hour,dest_weather_station,dest_weather_date,dest_weather_source,dest_weather_latitude,dest_weather_longitude,dest_weather_elevation,dest_weather_name,dest_weather_report_type,dest_weather_quality_control,dest_weather_wnd,dest_weather_cig,dest_weather_vis,dest_weather_tmp,dest_weather_dew,dest_weather_slp,dest_weather_ga1,dest_weather_ge1,dest_weather_gf1,dest_weather_ma1,dest_weather_rem,dest_weather_gd1,dest_weather_aa1,dest_weather_airport_code,dest_weather_hour
2015,1,1,1,4,2015-01-01,AA,19805,AA,N3HHAA,1693,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,1050,1044,-6.0,0.0,0.0,-1,1000-1059,13.0,1057,1404,5.0,1420,1409,-11.0,0.0,0.0,-1,1400-1459,0.0,null,0.0,150.0,145.0,127.0,1.0,1005.0,5,null,null,null,null,null,"Orlando, FL",America/New_York,America/Chicago,2015-01-01T16:00:00.000+0000,2015-01-01T13:00:00.000+0000,72530094846,2015-01-01T13:51:00.000+0000,7,41.995,-87.9336,201.8,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",FM-15,V030,"240,5,N,0057,5","22000,5,9,N","016093,5,N,5","-0094,5","-0144,5","10186,5","04,5,+07620,5,99,9","9,AGL ,+99999,+99999","04,99,5,99,9,99,9,07620,1,99,9,99,9","10173,5,09928,5",MET10401/01/15 07:51:02 METAR KORD 011351Z 24011G20KT 10SM SCT250 M09/M14 A3004 RMK AO2 SLP186 T10941144 (EPA),"2,99,1,+07620,5,9","01,0000,9,5",ORD,2015-01-01T13:00:00.000+0000,72205012815,2015-01-01T13:10:00.000+0000,7,28.4339,-81.325,27.4,"ORLANDO INTERNATIONAL AIRPORT, FL US",FM-16,V030,"360,5,N,0041,5","00396,5,M,N","016093,5,N,5","+0167,5","+0150,5","99999,9","07,5,+00396,5,99,9","9,AGL ,+99999,+99999","99,99,9,99,9,99,9,00396,1,99,9,99,9","10247,5,10208,5",MET11001/01/15 08:10:02 SPECI KMCO 011310Z 36008KT 10SM BKN013 BKN020 17/15 A3026 RMK AO2 RAE00 P0000 T01670150 (RK),"3,99,1,+00396,5,9","01,0000,2,1",MCO,2015-01-01T13:00:00.000+0000
2015,1,1,1,4,2015-01-01,AA,19805,AA,N3LWAA,2457,13930,1393003,30977,ORD,"Chicago, IL",IL,17,Illinois,41,14771,1477101,32457,SFO,"San Francisco, CA",CA,6,California,91,2010,2011,1.0,1.0,0.0,0,2000-2059,12.0,2023,2233,5.0,2250,2238,-12.0,0.0,0.0,-1,2200-2259,0.0,null,0.0,280.0,267.0,250.0,1.0,1846.0,8,null,null,null,null,null,"San Francisco, CA",America/Los_Angeles,America/Chicago,2015-01-02T02:00:00.000+0000,2015-01-01T23:00:00.000+0000,72530094846,2015-01-01T23:51:00.000+0000,7,41.995,-87.9336,201.8,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",FM-15,V030,"240,5,N,0072,5","02134,5,M,N","016093,5,N,5","-0011,5","-0094,5","10179,5","08,5,+02134,5,99,9","9,AGL ,+99999,+99999","99,99,9,99,9,99,9,02134,1,99,9,99,9","10169,5,09925,

# Next steps
- Join weather data for ALL cities, not just Atlanta, Chicago
- Weather data should be joined at 2 hour lag
- Change time to at least 2 hours before

# Features
- How many delays at airport before this flight?
- Incoming flight delay
- Weather at departure station and arrival station


# Building Model
- Random Forest
- Logistic Regression

## EDA

In [0]:
weather_airline_joined.printSchema()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-945005255680342> in <module> 
 ----> 1 weather_airline_joined . printSchema ( ) 

 NameError : name 'weather_airline_joined' is not defined

In [0]:
delay_columns = ['DEP_DELAY','DEP_DEL15']
delay = weather_airline_joined.select(*delay_columns)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-945005255680343> in <module> 
 1 delay_columns = [ 'DEP_DELAY' , 'DEP_DEL15' ] 
 ----> 2 delay = weather_airline_joined . select ( * delay_columns ) 

 NameError : name 'weather_airline_joined' is not defined

In [0]:
delay.describe().show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-945005255680344> in <module> 
 ----> 1 delay . describe ( ) . show ( ) 

 NameError : name 'delay' is not defined

In [0]:
def plot_hist(labels,values):
    df = pd.DataFrame({'lab':labels, 'val':values})
    df.plot.bar(x='lab', y='val', rot=0)

In [0]:
# We will use filter instead of rdd, and take advantage of predicate pushdown
import math
def makeHistogram(_min,_max,numBuckets,colName):
    _range = list(range(math.floor(_min), math.ceil(_max), round((abs(_min)+abs(_max))/numBuckets)))
    _counts = np.zeros(len(_range))
    for idx, val in enumerate(_range):
        if idx < len(_range)-1:
            _counts[idx] = delay.filter(F.col(colName) >= _range[idx]) \
                               .filter(F.col(colName) <= _range[idx+1]) \
                               .count()
    plot_hist(_range,_counts)

In [0]:
%%time
display(makeHistogram(-28.0,1221,11,'DEP_DELAY'))

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <timed eval> in <module> 

 <command-945005255680346> in makeHistogram (_min, _max, numBuckets, colName) 
 2 import math
 3 def makeHistogram ( _min , _max , numBuckets , colName ) : 
 ----> 4 _range = list ( range ( math . floor ( _min ) , math . ceil ( _max ) , round ( ( abs ( _min ) + abs ( _max ) ) / numBuckets ) ) ) 
 5 _counts = np . zeros ( len ( _range ) ) 
 6 for idx , val in enumerate ( _range ) : 

 /databricks/spark/python/pyspark/sql/functions.py in _ (col) 
 66 def _ ( col ) : 
 67 sc = SparkContext . _active_spark_context
 ---> 68 jc = getattr ( sc . _jvm . functions , name ) ( _to_java_column ( col ) ) 
 69 return Column ( jc ) 
 70 _ . __name__ = name

 /databricks/spark/python/pyspark/sql/column.py in _to_java_column (col) 
 54 "{0} of type {1}. " 
 55 "For column literals, use 'lit', 'array', 'struct' or 'create_map' " 
 ---> 56 "function.".format(col, type(col)))
 57 return jcol
 58 

 TypeError : Invalid argument, not a string or column: -28.0 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [0]:
delay_histogram = delay.select('DEP_DELAY').rdd.flatMap(lambda x: x).histogram(11)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-945005255680348> in <module> 
 ----> 1 delay_histogram = delay . select ( 'DEP_DELAY' ) . rdd . flatMap ( lambda x : x ) . histogram ( 11 ) 

 NameError : name 'delay' is not defined

In [0]:
# Loading the Computed Histogram into a Pandas Dataframe for plotting
pd.DataFrame(
    list(zip(*delay_histogram)), 
    columns=['bin', 'frequency']
).set_index(
    'bin'
).plot(kind='bar');


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-945005255680349> in <module> 
 1 # Loading the Computed Histogram into a Pandas Dataframe for plotting 
 ----> 2 pd.DataFrame(
 3 list ( zip ( * delay_histogram ) ) , 
 4 columns = [ 'bin' , 'frequency' ] 
 5 ) . set_index ( 

 NameError : name 'pd' is not defined

# Stations

In [0]:
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/DEMO8/gsod/stations.csv.gz")

In [0]:
display(stations.describe())

summary,usaf,wban,name,country,state,call,lat,lon,elev,begin,end
count,29771,29771,28982,28833,6742,10887,28567,28566,28473,29771,29771
mean,571613.0088459856,89988.97413590406,NaN,null,null,6504.5,30.674006125949553,-3.551033956451764,344.5273416921254,1.9782414351818886E7,2.004958363887676E7
stddev,304536.39977816143,27277.1013211369,NaN,null,null,9198.752116455797,28.80919467376261,87.29020816944056,590.7682889480685,234526.86689842667,193681.5670540804
min,007018,00001,...,AA,AK,050E,-0.017,-0.005,+0000.0,19010101,19051231
max,A51256,99999,ZYRYANKA,ZI,YT,ZYYY,9.994,99.983,-0999.9,20190103,20190305


In [0]:
from pyspark.sql import functions as f
stations.where(f.col('name').contains('ATLANTA HARTSFIELD INTERNATIONAL AIRPORT, GA US'))

Out[54]: DataFrame[usaf: string, wban: string, name: string, country: string, state: string, call: string, lat: string, lon: string, elev: string, begin: string, end: string]

In [0]:
display(stations.where(f.col('call').contains('LAX')))


usaf,wban,name,country,state,call,lat,lon,elev,begin,end
999999,23174,LOS ANGELES MUNICIPAL ARPT,US,CA,KLAX,33.938,-118.389,+0099.4,19470101,19721231
722950,23174,LOS ANGELES INTERNATIONAL AIR,US,CA,KLAX,33.938,-118.389,+0029.6,19440101,20190304


In [0]:
stations.select('name').distinct().count()

Out[56]: 26497

In [0]:
display(stations.select('name').distinct())

name
HATTFJELLDAL-KRUTA
COLLAFIRTH HILL
WINDY HEAD
ST ATHAN
ARESKUTAN
RYDAL
FLASH-IN-STAFFORD
EPINAL-DOGNEVILLE
FET I EIDFJORD
SAINT LEOCADIE


In [0]:
weather.select('NAME').distinct().count()

Out[58]: 14857

In [0]:
display(weather.select('name').distinct())

name
"KING KHALED AB, SA"
"MUIR ARMY AIR FIELD, PA US"
"KOLYMSKAYA, RS"
"BORISOGLEBSK, RS"
"GRIMSEL HOSPIZ, SZ"
"RENO STEAD AIRPORT, NV US"
"VITIM, RS"
"BRAVICEA, MD"
"SUNLIGHT, CO US"
"SOCHI, RS"
